In [3]:
!pip install --upgrade pip
!pip install pillow
!pip install scipy
#from PIL import Image
!pip install keras-tuner
!pip install pandas
!pip install scikit-learn

In [4]:
!apt install libgl1-mesa-glx -y
import cv2


Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgl1-mesa-glx is already the newest version (20.0.8-0ubuntu1~18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


ModuleNotFoundError: No module named 'cv2'

In [5]:
# First train
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from sklearn.utils import class_weight
import numpy as np
import datetime

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras import *
import tensorflow as tf 


#import tensorflow_addons as tfa
# dimensions of our images.

#import imgaug.augmenters as img_aug

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import SGD,Adam
import tensorflow
from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten, Dense, BatchNormalization, Dropout, GlobalMaxPooling2D
from tensorflow.keras.applications import DenseNet121,ResNet152V2,InceptionResNetV2
from keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import apply_affine_transform

def recall_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1_metric(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def get_train_image_quant(directory):
    datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator()
    data = datagen.flow_from_directory(directory)
    unique = np.unique(data.classes, return_counts=True)
    labels_dict = dict(zip(unique[0], unique[1]))
    print(labels_dict)
    image_quant_per_class = []
    for key in labels_dict:
        for i in range(labels_dict[key]):
            image_quant_per_class.append(key)
    return image_quant_per_class

def get_class_weights(labels, one_hot=False):
        if one_hot is False:
            n_classes = max(labels) + 1
        else:
            n_classes = len(labels[0])
        class_counts = [0 for _ in range(int(n_classes))]
        if one_hot is False:
            for label in labels:
                class_counts[label] += 1
        else:
            for label in labels:
                class_counts[label.index(1)] += 1
        return {i : (1. / class_counts[i]) * float(len(labels)) / float(n_classes) for i in range(int(n_classes))}






def add_noise(img):
    #print(img.shape)
    #seq = img_aug.Sequential([
    #img_aug.Emboss(alpha=(0.2, 0.2), strength=(1.5, 1.5))])
  #  cv2_imshow(img)
    angle = random.choice([0, 90, 180, 270])
    if angle != 0:
        img = apply_affine_transform(img, theta=angle, fill_mode='nearest')
    return img
    


def first_train(train_dir,val_dir,path_to_model):
    img_width, img_height = 400, 400

    train_data_dir = train_dir
    validation_data_dir = val_dir

    train_images_labels = get_train_image_quant(train_data_dir) # for weights balancing
    print (get_class_weights(train_images_labels))


    nb_train_samples = len(os.listdir(train_dir+"/clean/")) + len(os.listdir(train_dir+"/with_carotid/"))
    nb_validation_samples = len(os.listdir(val_dir+"/clean/")) + len(os.listdir(val_dir+"/with_carotid/"))

    epochs = 50
    batch_size = 16

    #path_to_model = "gdrive/MyDrive/data_sets/panoramic/panoramic_firsttrain.h5"

    if K.image_data_format() == 'channels_first':
        input_shape = (3, img_width, img_height)
    else:
        input_shape = (img_width, img_height, 3)

    train_datagen = ImageDataGenerator(
        # horizontal_flip=True,
        # vertical_flip=True,
         rescale=1. / 255,
      #   preprocessing_function=add_noise,
         width_shift_range=[0,40], 
         zoom_range=[0.9,1.1],
          rotation_range=15,
          shear_range=0.1,
           fill_mode='constant', cval=0
           
    )

# this is the augmentation configuration we will use for testing:
# only rescaling
    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

    validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')







    # top layer
    base_model = InceptionResNetV2(
            weights='imagenet',  # Load weights pre-trained on ImageNet.
            input_shape=input_shape,
            include_top=False)  # Do not include the ImageNet classifier at the top.
    base_model.trainable = False

    # top layer
    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(2, activation='softmax')(x)
    model = Model(inputs, outputs)
    
    #auc = tf.keras.metrics.AUC(name='auc')
    #auc = tfa.metrics.F1Score(num_classes=2,name='f1')
    opt = Adam(learning_rate=0.0001)
    
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy',f1_metric])

    print(model.summary())



    early = EarlyStopping(monitor='val_f1_metric', min_delta=0.01, \
        patience=10, verbose=1, mode='max')

    checkpoint = ModelCheckpoint(path_to_model, monitor='val_f1_metric',verbose=1,   # val_recall_metric
                                    save_best_only=True, save_weights_only=False, mode='max', save_freq='epoch')

    #log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    #tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,  histogram_freq=1, write_graph=True, write_images=True) 

 # 'val_accuracy'
    model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=300,#epochs,
    verbose=1,
    callbacks=[ early,checkpoint],
    validation_data=validation_generator,
    validation_steps=nb_validation_samples//batch_size,
    class_weight=get_class_weights(train_images_labels)
    )    

In [7]:
# fine tuning - Second train
import datetime
from tensorflow.keras.models import load_model
from sklearn.utils import class_weight
import numpy as np

def get_train_image_quant(directory):
    datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator()
    data = datagen.flow_from_directory(directory)
    unique = np.unique(data.classes, return_counts=True)
    labels_dict = dict(zip(unique[0], unique[1]))
    print(labels_dict)
    image_quant_per_class = []
    for key in labels_dict:
        for i in range(labels_dict[key]):
            image_quant_per_class.append(key)
    return image_quant_per_class

def get_class_weights(labels, one_hot=False):
        if one_hot is False:
            n_classes = max(labels) + 1
        else:
            n_classes = len(labels[0])
        class_counts = [0 for _ in range(int(n_classes))]
        if one_hot is False:
            for label in labels:
                class_counts[label] += 1
        else:
            for label in labels:
                class_counts[label.index(1)] += 1
        return {i : (1. / class_counts[i]) * float(len(labels)) / float(n_classes) for i in range(int(n_classes))}

def fine_tuning_train(train_dir,val_dir,previous_model_path, current_model_path):
    img_width, img_height = 400, 400

    train_data_dir = train_dir
    validation_data_dir = val_dir

    train_images_labels = get_train_image_quant(train_data_dir) # for weights balancing
    print (get_class_weights(train_images_labels))


    nb_train_samples = len(os.listdir(train_dir+"/clean/")) + len(os.listdir(train_dir+"/with_carotid/"))
    nb_validation_samples = len(os.listdir(val_dir+"/clean/")) + len(os.listdir(val_dir+"/with_carotid/"))

    epochs = 50
    batch_size = 12

     

    train_images_labels = get_train_image_quant(train_data_dir) # for weights balancing
    print (get_class_weights(train_images_labels))


#layer_to_train = 3
#for layer in model.layers[:layer_to_train]:
#        if not isinstance(layer, BatchNormalization):
#            layer.trainable = True  

    dependencies = {
            'recall_metric': recall_metric,
            "f1_metric": f1_metric
        }

    model = load_model(previous_model_path, custom_objects=dependencies)


    model.trainable = True
    #model.layers[-3:].trainable = True

    opt =  tensorflow.keras.optimizers.Adam(1e-6)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy',f1_metric])


    print(model.summary())

    train_datagen = ImageDataGenerator(
        # horizontal_flip=True,
        # vertical_flip=True,
         rescale=1. / 255,
      #   preprocessing_function=add_noise,
         width_shift_range=[0,40], 
         zoom_range=[0.9,1.1],
          rotation_range=15,
          shear_range=0.1,
           fill_mode='constant', cval=0
           
    )

# this is the augmentation configuration we will use for testing:
# only rescaling
    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

    validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')



    early = EarlyStopping(monitor='val_loss', min_delta=0.0001, \
        patience=25, verbose=1, mode='auto')

    checkpoint = ModelCheckpoint(current_model_path, monitor='val_accuracy',verbose=1,   # val_recall_metric
                                    save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')

    log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,  histogram_freq=1, write_graph=True, write_images=True) 

#class_weights = dict(zip(np.unique(train_generator.classes), class_weight.compute_class_weight('balanced', np.unique(train_generator.classes), train_generator.classes))) 




    model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=300,#epochs,
    verbose=1,
    callbacks=[ early,checkpoint],
    validation_data=validation_generator,
    validation_steps=nb_validation_samples//batch_size,
    class_weight=get_class_weights(train_images_labels)
    )  

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.imagenet_utils import decode_predictions
# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.models import load_model
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import tensorflow
from sklearn.metrics import precision_score,recall_score,f1_score,confusion_matrix,accuracy_score

def recall_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = tensorflow.keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = tensorflow.keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tensorflow.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tensorflow.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tensorflow.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tensorflow.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tensorflow.newaxis]
    heatmap = tensorflow.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tensorflow.maximum(heatmap, 0) / tensorflow.math.reduce_max(heatmap)
    return heatmap.numpy()

#map grad overly
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))


def prepare_img(img, img_width, img_height,channels):
    img = img_to_array(img)
    img = img.reshape(1, img_width, img_height, channels).astype('float32')
    img = img / 255.0
    return img

def calc_results_onTest(Testpath,modelPath,fold_num):
    channels =3
    img_width, img_height = 400, 400



    dependencies = {
            'recall_metric': recall_metric,
            "f1_metric": f1_metric
        }

    model = load_model(modelPath, custom_objects=dependencies)

    model1 = model

# Remove last layer's softmax
    #model1.layers[-1].activation = None

# Print what the top predicted class is

#print("Predicted:", decode_predictions(preds, top=1)[0])
# Generate class activation heatmap
    path = Testpath

    y_preds,y_true = [],[]
    for class_type in ["with_carotid/","clean/"]:
      file_list = os.listdir(path+class_type)


  
      counter=0
      for filename in file_list:
    #print("open",filename )
        counter+=1
        img = load_img(path+class_type+filename, grayscale=False, target_size=(img_width, img_height))
    #plt.matshow(img)
    #plt.show()
        img = prepare_img(img,img_width, img_height,channels)
        preds = model1.predict(img)
        print(counter," ", filename,np.argmax(preds))
        y_preds.append(np.argmax(preds))
        if "with_carotid" in class_type:y_true.append(1)
        else: y_true.append(0)
    #heatmap = make_gradcam_heatmap(img, model1.get_layer('densenet121'), 'conv5_block16_concat',1)

# Display heatmap
    #plt.matshow(heatmap)
    #plt.show()

    #save_and_display_gradcam(path+class_type+filename, heatmap)
    if os.path.exists("output_CV_01.txt"):
        f=open("output_CV_01.txt", 'a')
    else:
        f=open("output_CV_01.txt", 'w+')
    



    f.write("\n")
    f.write("---------------------"+str(fold_num)+"---------------------"+"\n")
    f.write("y_preds: "+ str(y_preds)+"\n")
    f.write("y_true" + str(y_true)+"\n")

    f.write("recall: " + str(recall_score(y_true, y_preds))+"\n")
    f.write("precision: "+ str(precision_score(y_true, y_preds))+"\n")
    f.write("F1: "+ str(f1_score(y_true, y_preds))+"\n")
    f.write("accuracy: "+ str(accuracy_score(y_true, y_preds))+"\n")
    tn, fp, fn, tp = confusion_matrix(y_true, y_preds).ravel()
    specificity = tn / (tn+fp)
    f.write("specificity: " +str(specificity)+"\n")
    f.write("confusion: "+"\n")
    f.write(str(confusion_matrix(y_true, y_preds))+"\n")
    f.write("------------------------------------------"+"\n")
    f.write("\n")
    f.close()
    return



In [ ]:
for i in range(1,3):
    print(str(i))
    train = "panoram/CV_7/fold_"+str(i)+"/train/"
    val =   "panoram/CV_7/fold_"+str(i)+"/val/"
    print(train)
    print(val)
    first_train(train,val, path_to_model="panoram/CV_7/CV7_"+str(i) +".h5")
    print("--------------------------------END FIRST TRAIN fold:"+str(i)+" ----------------------------------------------")
    fine_tuning_train(train,val, previous_model_path = "panoram/CV_7/CV7_"+str(i) +".h5", current_model_path = "panoram/CV_7/CV7_"+str(i) +".h5" )
    print("--------------------------------END SECOND fold:"+str(i)+" ----------------------------------------------")
    
    calc_results_onTest(Testpath="panoram/CV_7/fold_"+str(i)+"/test/",modelPath="panoram/CV_7/CV7_"+str(i) +".h5",fold_num=i)
    
    
    

1
panoram/CV_7/fold_1/train/
panoram/CV_7/fold_1/val/
Found 371 images belonging to 2 classes.
{0: 240, 1: 131}
{0: 0.7729166666666667, 1: 1.416030534351145}
Found 371 images belonging to 2 classes.
Found 144 images belonging to 2 classes.
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 400, 400, 3)]     0         
_________________________________________________________________
inception_resnet_v2 (Functio (None, 11, 11, 1536)      54336736  
_________________________________________________________________
flatten (Flatten)            (None, 185856)            0         
_________________________________________________________________
dense (Dense)                (None, 1024)              190317568 
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
___________________


Epoch 00008: val_accuracy did not improve from 0.77083
Epoch 9/300
30/30 [==============================] - 43s 1s/step - loss: 0.5387 - accuracy: 0.7744 - f1_metric: 0.7740 - val_loss: 0.5089 - val_accuracy: 0.7569 - val_f1_metric: 0.7569

Epoch 00009: val_accuracy did not improve from 0.77083
Epoch 10/300
30/30 [==============================] - 45s 1s/step - loss: 0.4993 - accuracy: 0.7660 - f1_metric: 0.7657 - val_loss: 0.4724 - val_accuracy: 0.7708 - val_f1_metric: 0.7708

Epoch 00010: val_accuracy did not improve from 0.77083
Epoch 11/300
30/30 [==============================] - 43s 1s/step - loss: 0.4590 - accuracy: 0.8078 - f1_metric: 0.8076 - val_loss: 0.4515 - val_accuracy: 0.7708 - val_f1_metric: 0.7708

Epoch 00011: val_accuracy did not improve from 0.77083
Epoch 12/300
30/30 [==============================] - 43s 1s/step - loss: 0.4690 - accuracy: 0.8050 - f1_metric: 0.8056 - val_loss: 0.4927 - val_accuracy: 0.7569 - val_f1_metric: 0.7569

Epoch 00012: val_accuracy did no


Epoch 00041: val_accuracy did not improve from 0.89583
Epoch 42/300
30/30 [==============================] - 43s 1s/step - loss: 0.1004 - accuracy: 0.9666 - f1_metric: 0.9667 - val_loss: 0.3929 - val_accuracy: 0.8681 - val_f1_metric: 0.8681

Epoch 00042: val_accuracy did not improve from 0.89583
Epoch 43/300
30/30 [==============================] - 43s 1s/step - loss: 0.0974 - accuracy: 0.9694 - f1_metric: 0.9694 - val_loss: 0.3666 - val_accuracy: 0.8889 - val_f1_metric: 0.8889

Epoch 00043: val_accuracy did not improve from 0.89583
Epoch 44/300
30/30 [==============================] - 45s 2s/step - loss: 0.0657 - accuracy: 0.9889 - f1_metric: 0.9889 - val_loss: 0.4233 - val_accuracy: 0.8542 - val_f1_metric: 0.8542

Epoch 00044: val_accuracy did not improve from 0.89583
Epoch 45/300
30/30 [==============================] - 42s 1s/step - loss: 0.0758 - accuracy: 0.9721 - f1_metric: 0.9722 - val_loss: 0.4074 - val_accuracy: 0.8750 - val_f1_metric: 0.8750

Epoch 00045: val_accuracy did n

Epoch 1/300
23/23 [==============================] - 57s 2s/step - loss: 2.1669 - accuracy: 0.5775 - f1_metric: 0.5688 - val_loss: 0.8066 - val_accuracy: 0.6667 - val_f1_metric: 0.6667

Epoch 00001: val_f1_metric improved from -inf to 0.66667, saving model to panoram/CV_7/CV7_2.h5
Epoch 2/300
23/23 [==============================] - 41s 2s/step - loss: 1.1555 - accuracy: 0.6620 - f1_metric: 0.6621 - val_loss: 0.4892 - val_accuracy: 0.8194 - val_f1_metric: 0.8194

Epoch 00002: val_f1_metric improved from 0.66667 to 0.81944, saving model to panoram/CV_7/CV7_2.h5
Epoch 3/300
23/23 [==============================] - 42s 2s/step - loss: 0.9194 - accuracy: 0.7070 - f1_metric: 0.6938 - val_loss: 1.1002 - val_accuracy: 0.5625 - val_f1_metric: 0.5625

Epoch 00003: val_f1_metric did not improve from 0.81944
Epoch 4/300
23/23 [==============================] - 39s 2s/step - loss: 0.8237 - accuracy: 0.7127 - f1_metric: 0.7228 - val_loss: 1.2423 - val_accuracy: 0.5833 - val_f1_metric: 0.5833

Epoch

Found 144 images belonging to 2 classes.
Epoch 1/300
30/30 [==============================] - 63s 2s/step - loss: 0.3917 - accuracy: 0.8747 - f1_metric: 0.8750 - val_loss: 0.5516 - val_accuracy: 0.7917 - val_f1_metric: 0.7917

Epoch 00001: val_accuracy improved from -inf to 0.79167, saving model to panoram/CV_7/CV7_2.h5
Epoch 2/300
30/30 [==============================] - 43s 1s/step - loss: 0.2855 - accuracy: 0.8969 - f1_metric: 0.8970 - val_loss: 0.5608 - val_accuracy: 0.8056 - val_f1_metric: 0.8056

Epoch 00002: val_accuracy improved from 0.79167 to 0.80556, saving model to panoram/CV_7/CV7_2.h5
Epoch 3/300
30/30 [==============================] - 44s 1s/step - loss: 0.2645 - accuracy: 0.8914 - f1_metric: 0.8917 - val_loss: 0.4991 - val_accuracy: 0.8403 - val_f1_metric: 0.8403

Epoch 00003: val_accuracy improved from 0.80556 to 0.84028, saving model to panoram/CV_7/CV7_2.h5
Epoch 4/300
30/30 [==============================] - 45s 1s/step - loss: 0.3031 - accuracy: 0.8830 - f1_metric

In [ ]:
!pip install pillow
!pip install scipy

#!pip install keras-tuner
!pip install pandas
!pip install scikit-learn
!pip install opencv-python

In [ ]:
#print(os.path.exists("./CV_output.txt"))
f=open("output_CV.txt", "r")
for l in f:
  print(l)
f.close()